# Inference using all  features

'weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove', 'esa'

Provide knowledge for training, and testing.

In [1]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from nltk.corpus import wordnet as wn
import os
from utils import prepare_dataset, rf_clf, write_to_file, eval_ensemble

# args
graph_path = "concepts_file/codc_graph/G_mindepth{}_codcdis{}_{}_maxconndist{}_{}.pickle"
concept_dial_path = "concepts_file/dial_mindepth{}_{}_{}.npy"
codc_path = "concepts_file/codc_mindepth{}_{}_mindis{}_{}.npy"
# candi_path = "attribute_file/candi_mindepth{}_codcdis{}_{}_{}_{}.npy"
# "attribute_file/candi_mindepth4_codcdis0_EMNLP_filter_nofilter_test.npy"
min_depth = 4
min_dist = 0
concept_filter_name = "EMNLP_filter"
graph_filter_weight = False
max_connect_dist = 4
if graph_filter_weight:
    graph_filter_name = "weightfilter"
else:
    graph_filter_name = "nofilter"
    

# cooccurance graph    
G = nx.read_gpickle(graph_path.format(min_depth, min_dist, 
            concept_filter_name, max_connect_dist, graph_filter_name))
# concepts and CODCs
codcs_train = np.load(
  codc_path.format(min_depth, concept_filter_name, min_dist, "train"),
  allow_pickle=True)
codcs_name_train = [[i[0] for i in item] for item in codcs_train]


codcs_test = np.load(
  codc_path.format(min_depth, concept_filter_name, min_dist, "test"),
  allow_pickle=True)
codcs_name_test = [[i[0] for i in item] for item in codcs_test]

# load attributes and parepare X, y
attrs_path = "attribute_file/attrs_mindepth{}_codcdis{}_{}_{}_{}.npy"

attrs_train = np.load(
  attrs_path.format(min_depth, min_dist, concept_filter_name, graph_filter_name, "train"),
  allow_pickle=True)

attrs_test = np.load(
  attrs_path.format(min_depth, min_dist, concept_filter_name, graph_filter_name, "test"),
  allow_pickle=True)

X, y = prepare_dataset(attrs_train, codcs_name_train, 
                       keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove', 'esa'])
X_test, y_test = prepare_dataset(attrs_test, codcs_name_test, 
                       keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove', 'esa'])


96620it [00:41, 2317.58it/s]
12083it [00:27, 441.18it/s]


In [2]:
# statistics of the dataset
print('number of positive examples:', len(np.where(np.array(y) == 1)[0]))
print('number of total examples:', len(y))
print('proportiona of positive example:', len(np.where(np.array(y) == 1)[0]) / len(y))

number of positive examples: 413191
number of total examples: 10603411
proportiona of positive example: 0.03896774349310802


In [3]:
X = np.array(X)
y = np.array(y)
np.random.seed(229)
# 准备1：10的比例
pos_X = X[y==1] 

pos_y = np.array([1 for item in pos_X])

len_neg = 10 * len(np.where(np.array(y) == 1)[0])

neg_ids = np.array([i for i, item in enumerate(y) if item == 0 ])
selected_index = np.random.permutation(len(neg_ids))[:len_neg]

neg_X = np.array(X)[neg_ids][selected_index]

neg_y = np.array([0 for item in neg_X])

X_tunable = np.vstack((pos_X, neg_X))
y_tunable = np.hstack((pos_y, neg_y))

print(X_tunable.shape, y_tunable.shape)

(4545101, 8) (4545101,)


In [4]:
# statistics of the dataset
print('number of positive examples:', len(np.where(np.array(y_tunable) == 1)[0]))
print('number of total examples:', len(y_tunable))
print('proportiona of positive example:', len(np.where(np.array(y_tunable) == 1)[0]) / len(y_tunable))

number of positive examples: 413191
number of total examples: 4545101
proportiona of positive example: 0.09090909090909091


# train using all (0.03)

In [5]:
clf_rf_10_all = rf_clf(X, y, X_test, y_test, depth=10, n_est=10)  

acc: 0.9883778820287581
recall: 0.6010523101571117 f1: 0.5820061787350028


In [13]:
clf_rf_12_all = rf_clf(X, y, X_test, y_test, depth=12, n_est=10)  

acc: 0.9877956928249548
recall: 0.6079346833511704 f1: 0.5831217162200245


In [31]:
clf_rf_8_all = rf_clf(X, y, X_test, y_test, depth=8, n_est=10)  

acc: 0.9886049101916659
recall: 0.5930740663891857 f1: 0.5778326223232333


In [32]:
ens_keys_8_all, ens_values_8_all = eval_ensemble(clf_rf_8_all, attrs_test, codcs_name_test,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove','esa'])


100%|██████████| 12083/12083 [00:57<00:00, 211.71it/s]


number of zeros hits 9188
topk 1 (0.2395928163535546, 0.04598833709257445, 0.0747029025243191)
number of zeros hits 7729
topk 2 (0.2035918232227096, 0.07755841502494554, 0.10758502832756274)
number of zeros hits 6758
topk 3 (0.17881872603382162, 0.10180105767568219, 0.12376051596356127)
number of zeros hits 6013
topk 4 (0.16183894728130432, 0.1227581010126127, 0.13290986966212298)
number of zeros hits 5471
topk 5 (0.1497144748820657, 0.14149681299973885, 0.1385802106003914)
number of zeros hits 3752
topk 10 (0.11326657287097575, 0.21261318336328924, 0.14174356018330658)


In [6]:
ens_keys_10_all, ens_values_10_all = eval_ensemble(clf_rf_10_all, attrs_test, codcs_name_test,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove','esa'])


100%|██████████| 12083/12083 [00:51<00:00, 236.64it/s]


number of zeros hits 9162
topk 1 (0.24174459985103036, 0.04633284072268595, 0.07537213008278207)
number of zeros hits 7626
topk 2 (0.20855747744765374, 0.07990298906772464, 0.11053084908891074)
number of zeros hits 6604
topk 3 (0.1856051201412453, 0.10644433211256554, 0.12882207026992232)
number of zeros hits 5858
topk 4 (0.1685425804849789, 0.12844766235474378, 0.13875780567804186)
number of zeros hits 5280
topk 5 (0.1557229164942481, 0.14822676038598018, 0.14460582719542736)
number of zeros hits 3648
topk 10 (0.11551766945295044, 0.21668903944853138, 0.1445420838769995)


In [14]:
ens_keys_12_all, ens_values_12_all = eval_ensemble(clf_rf_12_all, attrs_test, codcs_name_test,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove','esa'])


100%|██████████| 12083/12083 [00:53<00:00, 227.54it/s]


number of zeros hits 8978
topk 1 (0.25697260614085904, 0.049423208403070726, 0.08040591415116019)
number of zeros hits 7488
topk 2 (0.2153852520069519, 0.08239829744453402, 0.11409222181948314)
number of zeros hits 6461
topk 3 (0.1913707964357637, 0.10946355557166322, 0.1327398317272021)
number of zeros hits 5751
topk 4 (0.17199784821650252, 0.13100255953602413, 0.14160888395409607)
number of zeros hits 5235
topk 5 (0.15769262600347597, 0.14951010213733487, 0.14612698577000566)
number of zeros hits 3570
topk 10 (0.11754531159480262, 0.22043615258900712, 0.14710464238766724)


In [28]:
ens_keys_12_all_train, ens_values_12_all_train = eval_ensemble(clf_rf_12_all, attrs_train, codcs_name_train,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove','esa'])


100%|██████████| 96620/96620 [03:19<00:00, 484.64it/s]


number of zeros hits 29846
topk 1 (0.6910991513144277, 0.1423093704296902, 0.22722054989960105)
number of zeros hits 17017
topk 2 (0.6011436555578555, 0.24054688954950895, 0.3279377749191454)
number of zeros hits 11963
topk 3 (0.5299161664251708, 0.31194071027708253, 0.37388938141587924)
number of zeros hits 9254
topk 4 (0.47294210998413017, 0.36681889744989415, 0.3932046497338314)
number of zeros hits 7635
topk 5 (0.4264931346167116, 0.409540508750155, 0.39797117464314885)
number of zeros hits 4296
topk 10 (0.28502164016783144, 0.5342803358759058, 0.3565022711494449)


In [33]:
for topk in range(1, 11):
  write_to_file([item[:min(len(item), topk)] for item in ens_keys_8_all], 
              "knowledge/esa-all/rfdepth{}_mindepth{}_mindis{}_{}_{}_{}_top{}.txt"\
              .format(8, min_depth, min_dist, concept_filter_name, graph_filter_name, "test", topk))

In [29]:
for topk in range(1, 11):
  write_to_file([item[:min(len(item), topk)] for item in ens_keys_12_all_train], 
              "knowledge/train-knowledge/rfdepth{}_mindepth{}_mindis{}_{}_{}_{}_top{}.txt"\
              .format(12, min_depth, min_dist, concept_filter_name, graph_filter_name, "train", topk))

# train using 1:10

In [7]:
clf_rf_10 = rf_clf(X_tunable, y_tunable, X_test, y_test, depth=10, n_est=10)  

acc: 0.9727777041337298
recall: 0.6748247663304778 f1: 0.5605195909603534


In [8]:
ens_keys_10, ens_values_10 = eval_ensemble(clf_rf_10, attrs_test, codcs_name_test,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove','esa'])


100%|██████████| 12083/12083 [00:53<00:00, 227.97it/s]


number of zeros hits 9285
topk 1 (0.23156500868989488, 0.04415749468746874, 0.07189700496339381)
number of zeros hits 7832
topk 2 (0.19692957047090953, 0.07493198325146225, 0.10397352134410624)
number of zeros hits 6802
topk 3 (0.17708074705509116, 0.10120365726790163, 0.12278754813572684)
number of zeros hits 6058
topk 4 (0.1609699577919391, 0.12206325391372687, 0.13225892083086993)
number of zeros hits 5431
topk 5 (0.14996275759331293, 0.14194213285042162, 0.13896709612558372)
number of zeros hits 3696
topk 10 (0.11445005379458745, 0.21527850608918567, 0.14330220304601016)


In [9]:
clf_rf_8 = rf_clf(X_tunable, y_tunable, X_test, y_test, depth=8, n_est=10)  

acc: 0.975100636727216
recall: 0.6649549902415628 f1: 0.5627621917988898


In [10]:
clf_rf_12 = rf_clf(X_tunable, y_tunable, X_test, y_test, depth=12, n_est=10)  

acc: 0.9727206267608078
recall: 0.676681574896584 f1: 0.5609734162671635


In [11]:
ens_keys_8, ens_values_8 = eval_ensemble(clf_rf_8, attrs_test, codcs_name_test,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove', 'esa'])

100%|██████████| 12083/12083 [00:49<00:00, 243.81it/s]


number of zeros hits 9766
topk 1 (0.19175701398659273, 0.03642721885738326, 0.05928765623511767)
number of zeros hits 8225
topk 2 (0.17582554001489697, 0.06653320375315545, 0.0925313415073675)
number of zeros hits 7133
topk 3 (0.16262517586692046, 0.09206084562400679, 0.11231571845229213)
number of zeros hits 6344
topk 4 (0.15043863278987007, 0.11400135250580214, 0.1235944122764103)
number of zeros hits 5738
topk 5 (0.14049491020441945, 0.13260529140648752, 0.13003722315026298)
number of zeros hits 3869
topk 10 (0.11040304560125797, 0.20611000492914067, 0.13798021865211257)


In [12]:
ens_keys_12, ens_values_12 = eval_ensemble(clf_rf_12, attrs_test, codcs_name_test,
                                          keys=['weight', 'wup', 'path', 'lch', 'shortest_path_wn', 'w2v', 'glove', 'esa'])

100%|██████████| 12083/12083 [00:53<00:00, 227.57it/s]


number of zeros hits 9149
topk 1 (0.24282049159976826, 0.046581389152563525, 0.07581922361106404)
number of zeros hits 7616
topk 2 (0.20818505338078291, 0.07914360180591054, 0.10991468468862317)
number of zeros hits 6643
topk 3 (0.1836740323871003, 0.1043455260660644, 0.12701243722084737)
number of zeros hits 5924
topk 4 (0.16668046015062485, 0.12641781170456604, 0.13695081778824797)
number of zeros hits 5335
topk 5 (0.1542828767690143, 0.14612930383184897, 0.14302392097666855)
number of zeros hits 3587
topk 10 (0.11636183067119094, 0.2188339552224641, 0.14571977115167734)


In [ ]:
topk=6
write_to_file([item[:min(len(item), topk)] for item in ens_keys_12], 
              "knowledge/rfdepth{}_mindepth{}_mindis{}_{}_{}_{}_top{}.txt"\
              .format(12, min_depth, min_dist, concept_filter_name, graph_filter_name, "test", topk))